In [ ]:
# ==========================
# 0) Imports y configuración
# ==========================
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers, models, callbacks
from sklearn.pipeline import Pipeline

In [ ]:
# Reproducibilidad
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow:", tf.__version__)

NameError: name 'os' is not defined

In [ ]:
# =====================================
# 1) Cargar dataset Titanic (online URL)
# =====================================
df = pd.read_csv("/content/Breast_cancer_dataset.csv")

# Revisar columnas y forma
print(df.columns)
df.head()

NameError: name 'pd' is not defined

In [ ]:
df = df.drop(columns=['Unnamed: 32'], errors='ignore')

# Objetivo: diagnosis (benigno o maligno 0/1)
df['diagnosis'] = df['diagnosis'].map({'M':1, 'B':0})

y = df['diagnosis'].values

# Tomar todas las columnas numéricas menos 'id' y 'diagnosis'
X = df.drop(columns=['id','diagnosis']).copy()

In [ ]:
# Preprocesamiento con ColumnTransformer
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Ajustar y transformar todo el dataset
X_scaled = numeric_transformer.fit_transform(X)

In [ ]:
# Split train/test estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=SEED, stratify=y
)

In [ ]:
# Definir y compilar el modelo
def build_model(input_dim):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(4, activation='relu'),
        layers.Dropout(0.15),
        layers.Dense(2, activation='relu'),
        layers.Dropout(0.15),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model

model = build_model(X_train.shape[1])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 4)              │           124 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 137 (548.00 B)

 Trainable params: 137 (548.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Callbacks
cbs = [
    callbacks.EarlyStopping(monitor='val_auc', mode='max', patience=12, restore_best_weights=True),
    callbacks.ModelCheckpoint('breast_cancer_best.keras', monitor='val_auc', mode='max', save_best_only=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6)
]

In [ ]:
# Entrenamiento
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

hist = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=2,
    batch_size=32,
    callbacks=cbs,
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.4713 - auc: 0.5750 - loss: 0.7079 - val_accuracy: 0.2875 - val_auc: 0.7128 - val_loss: 0.6906 - learning_rate: 0.0010
Epoch 2/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4369 - auc: 0.6892 - loss: 0.6755 - val_accuracy: 0.2875 - val_auc: 0.8112 - val_loss: 0.6805 - learning_rate: 0.0010


In [ ]:
# Evaluación  en test
y_proba = model.predict(X_test).ravel()
y_pred = (y_proba > 0.5).astype(int)

print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred, digits=4))
print("\nAUC:", roc_auc_score(y_test, y_proba))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step

Matriz de confusión:
 [[  0 107]
 [  0  64]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       107
           1     0.3743    1.0000    0.5447        64

    accuracy                         0.3743       171
   macro avg     0.1871    0.5000    0.2723       171
weighted avg     0.1401    0.3743    0.2039       171


AUC: 0.8222838785046729


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Predecir
def predict_one(sample: dict) -> float:
    s = pd.DataFrame([sample])
    # Escalar con el mismo preprocesador
    s_proc = numeric_transformer.transform(s)
    proba = model.predict(s_proc).item()
    return proba

# Ejemplo
sample = X.iloc[0].to_dict()
proba = predict_one(sample)
print(f"\nProbabilidad de malignidad: {proba:.4f}")
print("Maligno" if proba >= 0.5 else "Benigno")

print(np.bincount(y_train))
print(np.bincount(y_test))

for i in range(10):
    sample = X.iloc[i].to_dict()
    proba = predict_one(sample)
    print(f"Fila {i} -> Prob malignidad: {proba:.4f}, {'Maligno' if proba>=0.5 else 'Benigno'}")

print(df['diagnosis'].value_counts())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

Probabilidad de malignidad: 0.6626
Maligno
[250 148]
[107  64]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Fila 0 -> Prob malignidad: 0.6626, Maligno
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Fila 1 -> Prob malignidad: 0.5564, Maligno
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Fila 2 -> Prob malignidad: 0.6360, Maligno
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Fila 3 -> Prob malignidad: 0.5877, Maligno
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Fila 4 -> Prob malignidad: 0.6183, Maligno
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Fila 5 -> Prob malignidad: 0.5072, Maligno
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Fila 6 -> Prob malignidad: 0.5032, Maligno
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Fila 7 -> Prob malignidad: 0.5015, Maligno
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Fila 8 -> Prob malignidad: 0.5135, Maligno
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Fila 9 -> Prob malignidad: 0.5015, Maligno
diagnosis
0    357
1    212
Name: count, dtype: int64
